# SEO Notebook (loads Pipulate functions)

In [ ]:
import notebook_finder
import goodsheet
from imp import reload

In [ ]:
if __name__ == '__main__':
    import pipulate
    reload(pipulate)
    import pipulate # Intentional, forces reliable updating of Pipulate functions
    func_dict = {x:eval('pipulate.%s' % x) for x in dir(pipulate) if x[0] is not '_'}
    worksheet = goodsheet.connect.open('Lookups').worksheet('Sheet1')
    end_range = worksheet.get_addr_int(worksheet.row_count, worksheet.col_count)
    cell_range = worksheet.range('A1:%s' % end_range)
    number_columns = worksheet.col_count
    col_names = []
    row_dict = {}
    for i, acell in enumerate(cell_range):
        row, col, val = acell.row, acell.col, acell.value
        print(i)
        if row == 1:
            col_names.append(val)
        else:
            row_dict[col_names[col-1]] = val
            if col%number_columns == 0:
                for key, val in row_dict.items():
                    if not val:
                        if key in [x for x in dir(pipulate) if x[0] is not '_']:
                            success_code, new_text = func_dict[key](**row_dict)
                            cell_address = row*number_columns-number_columns + col_names.index(key)
                            cell_range[cell_address].value = new_text
                row_dict = {}
    worksheet.update_cells(cell_range)